In [4]:
import csv
import pandas as pd
from sentence_transformers import SentenceTransformer
import numpy as np
from statistics import mean
from numpy.linalg import norm
from pathlib import Path

## Load Human Prompts

In [69]:
human_prompt_files = [
    "../sentence_embeddings/data/amazonTurkDesPrompt1.csv",
    "../sentence_embeddings/data/amazonTurkDesPrompt2.csv",
    "../sentence_embeddings/data/amazonTurkDesPrompt3.csv",
    "../sentence_embeddings/data/amazonTurkDesPrompt4.csv",
    "../sentence_embeddings/data/amazonTurkDesPrompt5.csv",
    "../sentence_embeddings/data/amazonTurkDesPrompt7.csv",
    "../sentence_embeddings/data/amazonTurkDesPrompt8.csv",
    "../sentence_embeddings/data/amazonTurkDesPrompt9.csv",
    "../sentence_embeddings/data/amazonTurkDesPrompt10.csv",
    "../sentence_embeddings/data/amazonTurkDesPrompt11.csv",
    "../sentence_embeddings/data/amazonTurkDesPrompt12.csv",
]

human_prompts = []

for human_prompt in human_prompt_files:
    prompts = []
    with open(human_prompt, 'r') as file:
        csvreader = csv.reader(file)
        for row in csvreader:
            prompts.append(row[0])
    prompts = [i.replace('\n',' ') for i in prompts]
    human_prompts.append(prompts)

In [70]:
pd.DataFrame(human_prompts)

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,ï»¿A jump rope that is also to be used as a ti...,A muscle stimulate band that can be worn over ...,"A simple, but strong band that has a hook to b...","A calf strengthener, similar to a finger stren...",A rubber ball that can be inflated / deflated ...,An x shaped resistance band that has 2 hand gr...,A mat that has electrosensitive pressure point...,A fold bike that could be used as a bike and a...,I think there should be a product that will he...,I am thinking of a portable exercise device.,...,A vest that makes you flex your chest muscles ...,A fold up tension devise that can fit into a s...,something very small that will allow you to wo...,A rowing type machine that can be folded up in...,This device is a dumbbell in which the weighte...,a small collapsible step that can be used to d...,How about an abdominal or muscle training MP3 ...,"Flexible, non-electric, ergonomic, compact","A collapsible, compact elliptical machine.",I am thinking of some sort of rubber band stro...
1,ï»¿A vest made of crystal quartz pieces connec...,Perhaps there could be something...a cord?...a...,a device which is attached to the joints so wh...,"When you blush, you put sensors on your cheeks...",People could wear kinetic bracelets and anklet...,solar equipment collects energy,A bicycle-powered television,have electrodes that can suction cup to our bo...,A battery that is charged by the energy from y...,A shoe that collects the energy generated when...,...,Im thinking of clothing that contians statice ...,I know skin conductance goes up when people ar...,A stationary bike with the wheel connected to ...,put pressure buttons in the bottoms of shoes a...,I am thinking of something simple like a watch...,sort of like roller blades that store the mech...,Energy from the motion of walking could be col...,A smart phone app that trackA smart phone app ...,"To collect energy from human motion, a station...",Power collecting shoes that joggers or top rac...
2,ï»¿with a candle that has marks on it,A clock or new redesigned clock that measures ...,waterfall clock tracking time by water ammount,Instead of relying on visual inputs these woul...,A small chart on your wrist,A giant hourglass,A clock that used binary to measure time.,Marking a piece of string with knots to measur...,A medication that lets you be so in tune with ...,a clock that acts more like a measure of the s...,...,"""Timeline"" is a flexible strip of LED lights e...",Digital self-drawn laser clock art. The design...,Use base 10 instead of base-6 numbers.,A block you put in the freezer and it turns bl...,We could divide time up into different color q...,The sun app. The sun app uses a satellite in ...,It would be the number of times my fan turned ...,A device that monitors the rotations and posit...,TIME IS INFINITE THEREFORE SOMETHING HAS TO BE...,light bulbs that dim as the sun goes down
3,"ï»¿""A tall cylindrical bottle","Like a party pop, this would be for celebratio...",a turn table which senses something underneath...,It would be a puffer kind of like a perfume bo...,Use the same container that weed killer comes ...,I thought of an automatic powdered sugar dispe...,Conveyor belt through a box that sprays the po...,A metal powder shaker with small circular hole...,An electrically powered sifter. Like a hand po...,A small long object that looks like a scanner....,...,A series of spinning rotors move several spray...,"This would be like a car wash, but instead of ...",a cannister that sprays and coats something of...,A small canister with a spigot at the top and ...,A container with a perforated lid to dispense ...,A small salt-shaker like item. This one has a ...,"A cup/tube cover, with multiple layers that ha...",I bake often at home and this prompt made me i...,A device that can be used while cooking and ba...,An automatic brush machine that could be adjus...
4,"ï»¿""A retractable gripper that has a central h...",A rotating shelf

## Load GPT Prompts

In [5]:
def get_all_files(directory, pattern):
    return [f for f in Path(directory).glob(pattern)]

In [9]:
gpt_files = get_all_files("../prompt_engineering/data/few_shot/", "*.csv")
len(gpt_files)

55

In [38]:
gpt_prompts = []

# for design_prompt_count in range(11):
for iteration_count in range(5):
    iteration_prompts = []
    # for iteration_count in range(5):
    for design_prompt_count in range(11):
        fail = False
        prompt = pd.read_csv(f"../prompt_engineering/data/few_shot/design_prompt_{design_prompt_count}_{iteration_count}.csv", sep=",")
        lst = []
        for d in prompt.values:
            lst.append(d[0])
        prompt = []
        for i in range(len(lst)):
            try:
                mod_statements = lst[i].replace(lst[i][:3], '')
                prompt.append(mod_statements)
            except AttributeError:
                fail = True
        if fail:
            print(f"Could not process design_prompt_{design_prompt_count}_{iteration_count}")

        iteration_prompts.append(prompt)
    gpt_prompts.append(iteration_prompts)

Could not process design_prompt_1_0
Could not process design_prompt_0_1
Could not process design_prompt_0_2
Could not process design_prompt_9_2
Could not process design_prompt_8_3
Could not process design_prompt_9_3
Could not process design_prompt_9_4
Could not process design_prompt_10_4


In [46]:
pd.DataFrame(gpt_prompts)

,0,1,2,3,4,5,6,7,8,9,10
0,[A collapsible set of weights that have a buil...,[A backpack that contains a motorized pulley s...,[Mechanically powered wall clock with a 24-hou...,"[A handheld device with a reservoir of powder,...","[""A rotary-style robotic arm mounted on the ce...",[Phone cases that have a built in strap so it ...,[A hands-free device that takes the clothes/to...,[Have the fountain mounted to the wall at an a...,[The measuring cup would be equipped with a sp...,[A knee brace that wraps around the joint and ...,[A lever-operated device with a cup-shaped cha...
1,[A core activator belt that can be worn to act...,[A generator that fits around the waist and us...,[An hourglass that measures the passage of tim...,"[Like a fog machine, this would be able to dis...","[A extendable ladder with a hook on the end., ...",[Having a device that works with smart watches...,[A foldable metal frame with two arms that can...,[Install a longer-length flexible drinking hos...,"[A tactile measuring cup with raised numbers.,...",[A metal structure with adjustable joints to i...,[A hammer or mallet that can be used to whack ...
2,[A handheld device with elastic bands and a ha...,[a small weighted wheel that rotates when you ...,"[mechanical gears clock that tracks time, robo...",[An automated robotic arm that moves around th...,[A motorized robotic arm that sits in the spac...,[Place signs in public areas warning pedestria...,[A laundry machine with inbuilt folding capabi...,[Make the fountain accessible to both wheelcha...,[The measuring cup has a setting to vibrate wh...,[A passive restraint device with straps and bu...,[A revolving blade mounted on a handle. This b...
3,[An adjustable resistance band that wraps arou...,[A device that attaches to your arms and legs ...,[A wristwatch that measures the passage of tim...,[It would be a spray bottle that sprays a fine...,[A telescoping pole with a basket on the end t...,[A text alert that reminds users to be aware o...,[Hand-cranked machine with flat surface and ar...,[Install a drinking fountain with disability-f...,[Provide a measuring cup with Braille identifi...,[A device that employs infrared sensors and ai...,[A metal clamp that has a small wheel on the s...
4,[A retractable jump rope that is stored in an ...,[A lightweight elastic fitness band that can b...,[A bracelet with tiny beads spaced out at each...,[A handheld device that has a nozzle that shoo...,[A telescopic “grabber” unit with an extendabl...,[Install sensors in the road to detect cell ph...,[An automated wall-mounted device that folds t...,[A built in water filter to provide clean drin...,[The measuring cup has a series of tactile mar...,[A device that uses an exoskeleton of rods and...,[A lever-based device that would use a pointed...


## Embed data

In [49]:
model = SentenceTransformer('all-MiniLM-L6-v2')

In [71]:
human_embeddings = [model.encode(prompt) for prompt in human_prompts]

gpt_embeddings_iterations = []
for iteration in gpt_prompts:
    gpt_embeddings_iterations.append([model.encode(prompt) for prompt in iteration])

In [53]:
np.array(gpt_embeddings_iterations).shape

C:\Users\grandid\AppData\Local\Temp\1\ipykernel_70588\2590467071.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  np.array(gpt_embeddings_iterations).shape


(5, 11)


# Similarity Metrics
## Nearest Generated Sample
Using cosine distance
Focus on design space coverage

In [54]:
distances_all = []

for embedding in gpt_embeddings_iterations:
    distances_model = []
    for count, generated_embedding in enumerate(embedding):
        distances = []
        for original_datapoint in human_embeddings[count]:
            min_distance = 1e9
            for generated_datapoint in generated_embedding:
                distance = np.dot(original_datapoint, generated_datapoint)/(norm(original_datapoint)*norm(generated_datapoint))     # TODO: this is similarity
                if distance < min_distance:
                    min_distance = distance
            distances.append(min_distance)
        mean_distance = mean(distances)
        # print(f"Mean distance of nearest generated samples for prompt {count} to human: {mean_distance}")
        distances_model.append(mean_distance)
    distances_all.append(distances_model)


In [55]:
distances_pd = pd.DataFrame(distances_all, index=range(5)).transpose()
distances_pd.style.background_gradient()

,0,1,2,3,4
0,0.185812,0.225916,0.218553,0.272062,0.174883
1,0.330108,0.391016,0.194984,0.348109,0.340478
2,0.126209,0.173583,0.133528,0.250715,0.191058
3,0.323045,0.273287,0.292489,0.384379,0.389031
4,0.463366,0.176385,0.263085,0.236072,0.425224
5,0.171115,0.219853,0.070793,0.133649,0.154825
6,0.380257,0.450574,0.332431,0.124407,0.493275
7,0.267855,0.088668,0.124275,0.320851,0.128599
8,0.214892,0.426518,0.405632,0.404761,0.409081
9,0.044297,0.048901,0.024089,0.145318,0.129883


In [56]:
distances_pd.mean(axis=1)

0     0.215445
1     0.320939
2     0.175019
3     0.332446
4     0.312826
5     0.150047
6     0.356189
7     0.186050
8     0.372177
9     0.078498
10    0.155446
dtype: float32

# Novelty/Diversity Metrics
## Convex hull hypervolume

In [58]:
from scipy.spatial import ConvexHull
from sklearn.decomposition import PCA
from tqdm import tqdm

In [73]:
np.array(human_embeddings).shape

(11, 100, 384)

In [77]:
np.array([human_embeddings] + gpt_embeddings_iterations).shape

C:\Users\grandid\AppData\Local\Temp\1\ipykernel_70588\549932370.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  np.array([human_embeddings] + gpt_embeddings_iterations).shape


(6, 11)

In [78]:
pca = PCA(n_components=3)

volumes_all = []
for prompts in tqdm([human_embeddings] + gpt_embeddings_iterations):
    volumes_prompts = []
    for prompt in tqdm(prompts, leave=False):
        embeddings = pca.fit_transform(prompt)
        convex_hull = ConvexHull(embeddings)
        volume_prompt = convex_hull.volume
        volumes_prompts.append(volume_prompt)
    volumes_all.append(volumes_prompts)

100%|██████████| 6/6 [00:00<00:00,  7.94it/s] 


In [81]:
convex_hull_volumes_df = pd.DataFrame(volumes_all, index=['human', 'iteration 1', 'iteration 2', 'iteration 3', 'iteration 4', 'iteration 5']).transpose()
convex_hull_volumes_df.style.background_gradient()

,human,iteration 1,iteration 2,iteration 3,iteration 4,iteration 5
0,0.249209,0.272238,0.185465,0.178628,0.144559,0.255028
1,0.243629,0.139101,0.136639,0.226110,0.115445,0.196254
2,0.321649,0.295191,0.132038,0.173457,0.191801,0.265659
3,0.317053,0.141965,0.190726,0.211800,0.139574,0.117120
4,0.338390,0.056472,0.349411,0.220953,0.220270,0.077826
5,0.330279,0.247635,0.245501,0.301907,0.265899,0.243826
6,0.276590,0.142680,0.064899,0.126344,0.294472,0.075083
7,0.327088,0.167231,0.373654,0.352700,0.132803,0.251590
8,0.306567,0.280266,0.091362,0.096986,0.112691,0.114206
9,0.254090,0.160218,0.114390,0.185052,0.071063,0.060806


In [82]:
convex_hull_volumes_df.mean(axis=0)

human          0.297031
iteration 1    0.178994
iteration 2    0.190398
iteration 3    0.197413
iteration 4    0.167923
iteration 5    0.157275
dtype: float64

In [83]:
convex_hull_volumes_df.mean(axis=1)

0     0.214188
1     0.176196
2     0.229966
3     0.186373
4     0.210554
5     0.272508
6     0.163345
7     0.267511
8     0.167013
9     0.140937
10    0.151306
dtype: float64